In [1]:
import os
import glob
import math
import time
import pandas as pd
import numpy as np
import scipy as sc
from sklearn.model_selection import KFold
import warnings
import matplotlib.pyplot as plt
import matplotlib
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader
import torch
import torch.nn as nn
import random
import seaborn as sns; sns.set_theme()
import torch.nn.functional as F
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from matplotlib.pyplot import figure
from IPython import display
from pandas.plotting import scatter_matrix
from sklearn.metrics import r2_score
from sklearn import svm
from numpy import std
from numpy import mean
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from matplotlib import cm
from sklearn.metrics import confusion_matrix

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
warnings.filterwarnings('ignore')
pd.set_option('max_columns', 300)

In [4]:
train_test_seurat = pd.read_csv('./integrate.csv')

In [5]:
train_test_seurat = train_test_seurat.T

In [6]:
train_test_seurat_std = train_test_seurat.std()
column_names = list(train_test_seurat.columns)
columns_remove = []
for i in range(train_test_seurat.shape[1]):
    if train_test_seurat_std[i] == 0:
        columns_remove.append(column_names[i])

In [7]:
train_test_seurat = train_test_seurat.drop(columns_remove, axis=1)

In [8]:
train_test_seurat[columns_remove[0]] = train_test_seurat.iloc[:, 0]

In [9]:
train_seurat = train_test_seurat.iloc[:90000, :]
test_seurat = train_test_seurat.iloc[90000:, :]

In [10]:
train = pd.read_csv('./MLR_Project_train.csv')
test = pd.read_csv('./MLR_Project_test.csv')

In [11]:
target = pd.DataFrame(pd.concat([train['TARGET'], test['TARGET']], axis= 0))

In [12]:
train_max = np.sum(train['TARGET'][train['TARGET']>0])
test_max = np.sum(test['TARGET'][test['TARGET']>0])

print('Maximum return of training set:', train_max)
print('Maximum return of testing set:', test_max)

Maximum return of training set: 195.6927566509
Maximum return of testing set: 55.96225182400002


In [13]:
reg = Ridge(alpha=0.5).fit(pd.DataFrame(train_seurat.iloc[:, :]), train['TARGET'])
pred = reg.predict(pd.DataFrame(train_seurat.iloc[:, :]))

pred_test = reg.predict(pd.DataFrame(test_seurat.iloc[:, :]))

train_res = np.sum(train['TARGET'][pred>0])
test_res = np.sum(test['TARGET'][pred_test>0])

In [14]:
print(f'Train naive random selection percentage return: {train_res/train_max*100}%')
print(f'Test naive random selection percentage return: {test_res/test_max*100}%')

Train naive random selection percentage return: 6.326525261528151%
Test naive random selection percentage return: 4.010962118285209%


### 1.3.1 Remove the Unnamed columns in dataframe

In [15]:
train = train.loc[:, ~train.columns.str.contains('^Unnamed')]
test = test.loc[:, ~test.columns.str.contains('^Unnamed')]

In [16]:
train_test_seurat.reset_index(inplace=True) 
train_test_seurat.drop("index",axis=1,inplace=True)
target.reset_index(inplace=True)
target.drop("index",axis=1,inplace=True)

In [17]:
train_test_seurat = pd.concat([train_test_seurat, target], axis=1)

In [18]:
train_test_seurat

,X39,X53,X45,X13,X36,X15,X51,X49,X33,X40,X31,X34,X32,X50,X30,X18,X46,X35,X37,X17,X42,X38,X43,X47,X0,X52,X48,X41,X2,X3,X25,X23,X9,X27,X14,X19,X24,X16,X29,X26,X28,X21,X1,X11,X20,X22,X5,X4,X6,X12,X7,X10,X8,X62,TARGET
0,-1.582156,1.230171,2.175619,2.246967,-0.755876,3.583055,0.098955,3.040176,0.919099,-0.889421,0.484578,0.481170,0.777112,-1.798247,0.873301,2.270459,2.912388,0.728549,-1.295910,2.275724,-0.359744,-0.363988,-0.717875,2.265473,1.019072,0.721157,2.260484,-1.834228,2.254474,2.254450,3.042155,2.259637,2.946272,2.258885,3.401480,3.345012,2.937407,3.343820,2.935077,2.941428,2.259203,2.938547,2.469231,2.945955,3.044228,2.259368,3.073369,2.943574,2.945749,2.259507,2.259501,3.071520,2.259504,-1.582156,0.013314
1,-1.584404,1.605045,-2.204271,2.227963,-0.760217,3.535377,0.300338,2.880602,0.943713,-0.890453,0.513255,0.509388,0.803151,-1.042563,0.900408,2.251683,2.831043,0.757081,-1.302729,2.256918,-0.368216,-0.372547,-0.725350,2.242051,1.084322,1.273463,2.239054,-1.836103,2.235786,2.235849,2.995526,2.240825,2.901227,2.240080,3.359956,3.304083,2.898066,3.302884,2.895731,2.902085,2.240400,2.910457,2.446440,2.903884,3.007216,2.240566,3.004351,2.907020,2.906517,2.240683,2.240706,3.025257,2.240732,-1.584404,-0.000448
2,-1.594548,1.172960,2.547328,2.138920,-0.779797,3.321969,-0.241806,2.693446,0.934271,-0.895068,0.505998,0.502247,0.793162,-1.196648,0.893682,2.165409,2.656538,0.750002,-1.334065,2.170502,-0.407147,-0.411875,-0.759699,2.150257,0.909735,0.858274,2.152557,-1.844565,2.150014,2.149977,2.786986,2.154385,2.734335,2.153668,3.172415,3.119236,2.720591,3.118004,2.718235,2.724605,2.154003,2.731983,2.340769,2.729945,2.798645,2.154173,2.832534,2.727687,2.728807,2.154322,2.154294,2.816114,2.154304,-1.594548,0.000244
3,-1.540117,1.471016,-1.464351,2.589700,-0.674728,4.499712,-0.070470,3.630669,-0.163725,-0.869522,0.771819,0.763805,1.057894,-1.013944,1.144550,2.601021,3.586176,1.014063,-1.175844,2.588606,-0.210578,-0.213300,-0.586264,2.586420,1.452937,1.030784,2.588699,-1.799160,2.583419,2.583483,3.941371,2.590836,3.676733,2.589976,4.174776,4.107204,3.672950,4.106144,3.670702,3.676991,2.590239,3.685219,2.879247,3.681201,3.949020,2.590387,3.953846,3.682269,3.681686,2.590519,2.590540,3.971859,2.590523,-1.540117,-0.000628
4,-1.601161,1.122772,2.557003,2.079653,-0.792563,3.184293,0.169903,2.380626,0.968678,-0.898042,0.546909,0.542502,0.829559,-1.363513,-0.469419,2.107699,2.530198,0.790754,-1.355026,2.112698,-0.433188,-0.438182,-0.782676,2.091026,0.925432,0.645674,2.091058,-1.850081,2.092565,2.092535,2.652600,2.096564,2.618092,2.095867,2.443792,2.998522,2.604873,2.997269,2.602504,2.608884,2.096211,2.615890,2.270373,2.613631,2.663541,2.096384,2.693932,2.611509,2.613064,2.096538,2.096525,2.682193,2.096533,-1.601161,0.003811
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109702,-1.811006,-2.112556,-0.131305,-1.445311,-1.197634,-0.818321,-1.049270,-1.122315,-0.508777,-0.357198,1.027011,-0.311548,0.721451,-2.989994,0.889711,-1.483919,-1.102665,-0.136718,0.872560,-1.485440,0.397786,-2.070832,0.489914,-1.491222,-4.525414,-2.391811,-1.488305,0.216442,-1.483623,-1.483645,-1.030355,-1.481442,-1.076202,-1.483214,-1.062814,-1.047183,-1.082408,-1.057290,-1.070795,-1.077186,-1.483106,-1.077155,-1.348169,-1.074557,-1.005825,-1.483853,-0.998264,-1.075246,-1.074719,-1.483446,-1.483451,-1.000564,-1.483459,-1.811006,0.000061
109703,0.753172,-1.087454,-0.172401,-0.671626,0.416608,-0.605980,-0.603986,-0.787102,1.405494,-0.960323,0.122744,1.482535,0.217549,-1.818208,-0.006719,-0.725304,-0.736942,1.349400,0.214743,-0.727687,-1.691140,0.832361,-1.892583,-0.731506,-3.272527,-1.266335,-0.731930,0.941092,-0.720566,-0.720532,-0.645284,-0.724744,-0.706717,-0.723241,-0.704043,-0.710849,-0.699841,-0.697351,-0.719490,-0.712830,-0.725970,-0.712226,-0.739703,-0.709396,-0.687081,-0.725374,-0.687902,-0.

In [19]:
train_test_seurat_ = pd.DataFrame()

for i in range(train_test_seurat.shape[1]-1):
    for j in range(train_test_seurat.shape[1]-1):
        train_test_seurat_[str(i)+'_'+str(j)+'_feat'] = train_test_seurat.iloc[:, i] * train_test_seurat.iloc[:, j]
        
train_test_seurat_target = pd.DataFrame(train_test_seurat['TARGET'])

train_test_seurat = train_test_seurat.drop(['TARGET'], axis = 1)

train_test_seurat = pd.concat([train_test_seurat, train_test_seurat_], axis = 1)

# train_test_seurat = (train_test_seurat-train_test_seurat.mean())/train_test_seurat.std()

train_test_seurat['TARGET'] = train_test_seurat_target

In [20]:
train_test_seurat

,X39,X53,X45,X13,X36,X15,X51,X49,X33,X40,X31,X34,X32,X50,X30,X18,X46,X35,X37,X17,X42,X38,X43,X47,X0,X52,X48,X41,X2,X3,X25,X23,X9,X27,X14,X19,X24,X16,X29,X26,X28,X21,X1,X11,X20,X22,X5,X4,X6,X12,X7,X10,X8,X62,0_0_feat,0_1_feat,0_2_feat,0_3_feat,0_4_feat,0_5_feat,0_6_feat,0_7_feat,0_8_feat,0_9_feat,0_10_feat,0_11_feat,0_12_feat,0_13_feat,0_14_feat,0_15_feat,0_16_feat,0_17_feat,0_18_feat,0_19_feat,0_20_feat,0_21_feat,0_22_feat,0_23_feat,0_24_feat,0_25_feat,0_26_feat,0_27_feat,0_28_feat,0_29_feat,0_30_feat,0_31_feat,0_32_feat,0_33_feat,0_34_feat,0_35_feat,0_36_feat,0_37_feat,0_38_feat,0_39_feat,0_40_feat,0_41_feat,0_42_feat,0_43_feat,0_44_feat,0_45_feat,0_46_feat,0_47_feat,0_48_feat,0_49_feat,0_50_feat,0_51_feat,0_52_feat,0_53_feat,1_0_feat,1_1_feat,1_2_feat,1_3_feat,1_4_feat,1_5_feat,1_6_feat,1_7_feat,1_8_feat,1_9_feat,1_10_feat,1_11_feat,1_12_feat,1_13_feat,1_14_feat,1_15_feat,1_16_feat,1_17_feat,1_18_feat,1_19_feat,1_20_feat,1_21_feat,1_22_feat,1_23_feat,1_24_feat,1_25_feat,1_26_feat,1_27_feat,1_28_feat,1_29_feat,1_30_feat,1_31_feat,1_32_feat,1_33_feat,1_34_feat,1_35_feat,1_36_feat,1_37_feat,1_38_feat,1_39_feat,1_40_feat,1_41_feat,...,51_13_feat,51_14_feat,51_15_feat,51_16_feat,51_17_feat,51_18_feat,51_19_feat,51_20_feat,51_21_feat,51_22_feat,51_23_feat,51_24_feat,51_25_feat,51_26_feat,51_27_feat,51_28_feat,51_29_feat,51_30_feat,51_31_feat,51_32_feat,51_33_feat,51_34_feat,51_35_feat,51_36_feat,51_37_feat,51_38_feat,51_39_feat,51_40_feat,51_41_feat,51_42_feat,51_43_feat,51_44_feat,51_45_feat,51_46_feat,51_47_feat,51_48_feat,51_49_feat,51_50_feat,51_51_feat,51_52_feat,51_53_feat,52_0_feat,52_1_feat,52_2_feat,52_3_feat,52_4_feat,52_5_feat,52_6_feat,52_7_feat,52_8_feat,52_9_feat,52_10_feat,52_11_feat,52_12_feat,52_13_feat,52_14_feat,52_15_feat,52_16_feat,52_17_feat,52_18_feat,52_19_feat,52_20_feat,52_21_feat,52_22_feat,52_23_feat,52_24_feat,52_25_feat,52_26_feat,52_27_feat,52_28_feat,52_29_feat,52_30_feat,52_31_feat,52_32_feat,52_33_feat,52_34_feat,52_35_feat,52_36_feat,52_37_feat,52_38_feat,52_39_feat,52_40_feat,52_41_feat,52_42_feat,52_43_feat,52_44_feat,52_45_feat,52_46_feat,52_47_feat,52_48_feat,52_49_feat,52_50_feat,52_51_feat,52_52_feat,52_53_feat,53_0_feat,53_1_feat,53_2_feat,53_3_feat,53_4_feat,53_5_feat,53_6_feat,53_7_feat,53_8_feat,53_9_feat,53_10_feat,53_11_feat,53_12_feat,53_13_feat,53_14_feat,53_15_feat,53_16_feat,53_17_feat,53_18_feat,53_19_feat,53_20_feat,53_21_feat,53_22_feat,53_23_feat,53_24_feat,53_25_feat,53_26_feat,53_27_feat,53_28_feat,53_29_feat,53_30_feat,53_31_feat,53_32_feat,53_33_feat,53_34_feat,53_35_feat,53_36_feat,53_37_feat,53_38_feat,53_39_feat,53_40_feat,53_41_feat,53_42_feat,53_43_feat,53_44_feat,53_45_feat,53_46_feat,53_47_feat,53_48_feat,53_49_feat,53_50_feat,53_51_feat,53_52_feat,53_53_feat,TARGET
0,-1.582156,1.230171,2.175619,2.246967,-0.755876,3.583055,0.098955,3.040176,0.919099,-0.889421,0.484578,0.481170,0.777112,-1.798247,0.873301,2.270459,2.912388,0.728549,-1.295910,2.275724,-0.359744,-0.363988,-0.717875,2.265473,1.019072,0.721157,2.260484,-1.834228,2.254474,2.254450,3.042155,2.259637,2.946272,2.258885,3.401480,3.345012,2.937407,3.343820,2.935077,2.941428,2.259203,2.938547,2.469231,2.945955,3.044228,2.259368,3.073369,2.943574,2.945749,2.259507,2.259501,3.071520,2.259504,-1.582156,2.503217,-1.946322,-3.442168,-3.555053,1.195914,-5.668951,-0.156562,-4.810032,-1.454158,1.407203,-0.766677,-0.761287,-1.229513,2.845107,-1.381699,-3.592219,-4.607851,-1.152678,2.050331,-3.600550,0.569170,0.575886,1.135790,-3.584332,-1.612331,-1.140982,-3.576438,2.902034,-3.566929,-3.566891,-4.813163,-3.575097,-4.661462,-3.573909,-5.381672,-5.292330,-4.647436,-5.290444,-4.643748,-4.653797,-3.574411,-4.649239,-3.906708,-4.660960,-4.816443,-3.574672,-4.862549,-4.657192,-4.660634,-3.574893,-3.574883,-4.859623,-3.574887,2.503217,-1.946322,1.513321,2.676383,2.764154,-0.929857,4.407770,0.121731,3.739936,1.130649,-1.094140,0.596113,0.591922,0.955981,-2.212152,1.074310,2.793052,3.582735,0.896240,-1.594190,2.799529,-0.442546,

In [21]:
# reg = Ridge(alpha=0.5).fit(pd.DataFrame(train_test_seurat.iloc[:90000, :-1]), train['TARGET'][:90000])

reg = LinearRegression().fit(train_test_seurat.iloc[:90000, :-1], train.iloc[:, -1])

train_pred = reg.predict(pd.DataFrame(train_test_seurat.iloc[:90000, :-1]))
test_pred = reg.predict(pd.DataFrame(train_test_seurat.iloc[90000:, :-1]))

train_res = np.sum(train['TARGET'][train_pred>0])
test_res = np.sum(test['TARGET'][test_pred>0])

In [22]:
print(train_res)
print(test_res)

38.8434845458
-1.2098240208000002


In [23]:
train

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,TARGET
0,5874.524387,1072.671848,41440.76212,41440.23732,70.405148,7.392780,70.377281,23229.69262,23229.72655,70.378864,7.389173,70.380160,23229.76782,23379.81637,83.418623,11.615135,83.418623,23466.72590,23466.72590,83.418623,7.425618,70.494241,23229.77107,23229.77107,70.376262,7.389056,70.376262,23229.77107,23229.77107,70.376262,4898.757333,15165.927590,297487.1654,297487.16540,15165.927590,4898.757333,70.376262,2.322977e+04,23229.77107,70.376262,7.389056,70.376262,23229.77107,23229.77107,64.192982,20.940618,76.070270,23376.73707,23343.13291,77.290965,347.308164,102.380501,24823.08137,24120.94894,332.757607,17.386711,129.622187,23936.99077,21670.192330,71.518948,11.399004,78.006816,26437.161240,23811.09670,141.997532,22.474794,0.013314
1,6124.154099,1072.802927,41440.76212,41442.22458,70.456758,7.356050,70.379576,23229.76020,23230.10472,70.273618,7.389813,70.329906,23229.46908,23384.98219,83.418623,11.615135,83.418623,23466.72590,23466.72590,83.418623,7.441223,70.702624,23229.77107,23229.77107,70.376262,7.389056,70.376262,23229.77107,23229.77107,70.376262,5100.350569,15788.076830,308790.4312,308790.43120,15788.076830,5100.350569,70.376262,2.322977e+04,23229.77107,70.376262,7.389056,70.376262,23229.77107,23229.77107,2.937218,-14.428210,75.204973,23317.46049,23309.42032,74.880368,689.670872,127.070357,36746.26762,31012.79786,-310.576721,4.532673,-47.045260,22053.23653,14626.733390,48.124991,-99.618253,-115.120518,7705.543821,22665.35143,-377.287072,-73.700375,-0.000448
2,5905.732593,1072.802927,41443.14358,41442.30403,70.422472,7.417794,70.376448,23229.48142,23229.61008,70.474265,7.388979,70.397301,23229.83396,23387.39168,83.418623,11.615135,83.418623,23466.72590,23466.72590,83.418623,7.440201,70.686178,23229.77107,23229.77107,70.376262,7.389056,70.376262,23229.77107,23229.77107,70.376262,5308.548086,16430.607950,320463.9968,320463.99680,16430.607950,5308.548086,70.376262,2.322977e+04,23229.77107,70.376262,7.389056,70.376262,23229.77107,23229.77107,141.231442,25.855392,75.184756,23248.26780,23307.91084,74.635569,634.523047,71.705965,28917.00549,24632.17456,419.071308,7.403187,118.846496,23430.24573,31251.552920,71.535567,53.482719,106.179152,37586.677270,23251.62576,261.098973,22.565621,0.000244
3,6029.325221,1072.802927,41440.76212,41442.25682,70.458533,7.366031,70.379221,23230.08433,23229.87971,70.288944,7.390120,70.370247,23229.81662,23390.02296,83.418623,11.615135,83.418623,23233.34325,23466.72590,83.418623,7.439191,70.692085,23229.77107,23229.77107,70.376262,7.389056,70.376262,23229.77107,23229.77107,70.376262,5510.265781,17053.141280,331774.2409,75710.89648,17053.141280,5510.265781,70.376262,2.322977e+04,23229.77107,70.376262,7.389056,70.376262,23229.77107,23229.77107,-1.580478,-7.499746,75.184756,23248.26836,23307.91084,74.635569,583.001082,70.384022,25642.94219,23482.26269,-249.671869,7.388974,49.809681,23193.67720,15867.015790,70.369496,-12.169114,63.930236,10052.351290,23229.64352,-10.549985,4.656636,-0.000628
4,6142.360146,1072.802927,41443.14358,41442.46480,70.413623,7.411287,70.376788,23229.70975,23229.82255,70.467206,7.389910,70.386986,23229.87603,23392.66710,70.572881,11.615135,83.418623,23466.72590,23466.72590,83.418623,7.438244,70.680386,23229.77107,23229.77107,70.376262,7.389056,70.376262,23229.77107,23229.77107,70.376262,1152.667997,17699.014450,343508.5253,343508.52530,17699.014450,5719.546213,70.376262,2.322977e+04,23229.77107,70.376262,7.389056,70.376262,23229.77107,23229.77107,149.144800,26.789196,74.904988,23229.77108,23260.32245,70.506980,566.549008,120.694073,25765.82131,24618.79392,363.188022,7.389057,78.826922,23235.86490,31790.064250,120.694068,42.971377,145.572170,37109.895810,24143.94971,188.639704,31.863254,0.003811
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [24]:
train_ = pd.DataFrame()

for i in range(train.shape[1]-1):
    for j in range(train.shape[1]-1):
        train_[str(i)+'_'+str(j)+'_feat'] = np.log(np.square(train.iloc[:, i] * train.iloc[:, j]))
        
train_target = pd.DataFrame(train['TARGET'])

train = train.drop(['TARGET'], axis = 1)

train = pd.concat([train, train_], axis = 1)

# train = (train-train.mean())/train.std()

train['TARGET'] = train_target

In [25]:
train

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,0_0_feat,0_1_feat,0_2_feat,0_3_feat,0_4_feat,0_5_feat,0_6_feat,0_7_feat,0_8_feat,0_9_feat,0_10_feat,0_11_feat,0_12_feat,0_13_feat,0_14_feat,0_15_feat,0_16_feat,0_17_feat,0_18_feat,0_19_feat,0_20_feat,0_21_feat,0_22_feat,0_23_feat,0_24_feat,0_25_feat,0_26_feat,0_27_feat,0_28_feat,0_29_feat,0_30_feat,0_31_feat,0_32_feat,0_33_feat,0_34_feat,0_35_feat,0_36_feat,0_37_feat,0_38_feat,0_39_feat,0_40_feat,0_41_feat,0_42_feat,0_43_feat,0_44_feat,0_45_feat,0_46_feat,0_47_feat,0_48_feat,0_49_feat,0_50_feat,0_51_feat,0_52_feat,0_53_feat,0_54_feat,0_55_feat,0_56_feat,0_57_feat,0_58_feat,0_59_feat,0_60_feat,0_61_feat,0_62_feat,0_63_feat,0_64_feat,0_65_feat,1_0_feat,1_1_feat,1_2_feat,1_3_feat,1_4_feat,1_5_feat,1_6_feat,1_7_feat,1_8_feat,1_9_feat,1_10_feat,1_11_feat,1_12_feat,1_13_feat,1_14_feat,1_15_feat,1_16_feat,1_17_feat,...,63_49_feat,63_50_feat,63_51_feat,63_52_feat,63_53_feat,63_54_feat,63_55_feat,63_56_feat,63_57_feat,63_58_feat,63_59_feat,63_60_feat,63_61_feat,63_62_feat,63_63_feat,63_64_feat,63_65_feat,64_0_feat,64_1_feat,64_2_feat,64_3_feat,64_4_feat,64_5_feat,64_6_feat,64_7_feat,64_8_feat,64_9_feat,64_10_feat,64_11_feat,64_12_feat,64_13_feat,64_14_feat,64_15_feat,64_16_feat,64_17_feat,64_18_feat,64_19_feat,64_20_feat,64_21_feat,64_22_feat,64_23_feat,64_24_feat,64_25_feat,64_26_feat,64_27_feat,64_28_feat,64_29_feat,64_30_feat,64_31_feat,64_32_feat,64_33_feat,64_34_feat,64_35_feat,64_36_feat,64_37_feat,64_38_feat,64_39_feat,64_40_feat,64_41_feat,64_42_feat,64_43_feat,64_44_feat,64_45_feat,64_46_feat,64_47_feat,64_48_feat,64_49_feat,64_50_feat,64_51_feat,64_52_feat,64_53_feat,64_54_feat,64_55_feat,64_56_feat,64_57_feat,64_58_feat,64_59_feat,64_60_feat,64_61_feat,64_62_feat,64_63_feat,64_64_feat,64_65_feat,65_0_feat,65_1_feat,65_2_feat,65_3_feat,65_4_feat,65_5_feat,65_6_feat,65_7_feat,65_8_feat,65_9_feat,65_10_feat,65_11_feat,65_12_feat,65_13_feat,65_14_feat,65_15_feat,65_16_feat,65_17_feat,65_18_feat,65_19_feat,65_20_feat,65_21_feat,65_22_feat,65_23_feat,65_24_feat,65_25_feat,65_26_feat,65_27_feat,65_28_feat,65_29_feat,65_30_feat,65_31_feat,65_32_feat,65_33_feat,65_34_feat,65_35_feat,65_36_feat,65_37_feat,65_38_feat,65_39_feat,65_40_feat,65_41_feat,65_42_feat,65_43_feat,65_44_feat,65_45_feat,65_46_feat,65_47_feat,65_48_feat,65_49_feat,65_50_feat,65_51_feat,65_52_feat,65_53_feat,65_54_feat,65_55_feat,65_56_feat,65_57_feat,65_58_feat,65_59_feat,65_60_feat,65_61_feat,65_62_feat,65_63_feat,65_64_feat,65_65_feat,TARGET
0,5874.524387,1072.671848,41440.76212,41440.23732,70.405148,7.392780,70.377281,23229.69262,23229.72655,70.378864,7.389173,70.380160,23229.76782,23379.81637,83.418623,11.615135,83.418623,23466.72590,23466.72590,83.418623,7.425618,70.494241,23229.77107,23229.77107,70.376262,7.389056,70.376262,23229.77107,23229.77107,70.376262,4898.757333,15165.927590,297487.1654,297487.16540,15165.927590,4898.757333,70.376262,2.322977e+04,23229.77107,70.376262,7.389056,70.376262,23229.77107,23229.77107,64.192982,20.940618,76.070270,23376.73707,23343.13291,77.290965,347.308164,102.380501,24823.08137,24120.94894,332.757607,17.386711,129.622187,23936.99077,21670.192330,71.518948,11.399004,78.006816,26437.161240,23811.09670,141.997532,22.474794,34.713522,31.312576,38.620801,38.620776,25.865294,21.357769,25.864502,37.463134,37.463137,25.864547,21.356792,25.864584,37.463140,37.476018,26.204504,22.261379,26.204504,37.483438,37.483438,26.204504,21.366633,25.867823,37.463141,37.463141,25.864473,21.356761,25.864473,37.463141,37.463141,25.864473,34.350234,36.610374,42.563013,42.563013,36.610374,34.350234,25.864473,37.463141,37.463141,25.864473,21.356761,25.864473,37.463141,37.463141,25.680549,23.440142,26.020076,37.475754,37.472877,26.051915,29.057186,26.614153,37.595819,37.538433,28.971589,23.068173,27.086009,37.523121,37.324147,25.896686,22.223813,26.070353,37.721813

In [26]:
test_ = pd.DataFrame()

for i in range(test.shape[1]-1):
    for j in range(test.shape[1]-1):
        test_[str(i)+'_'+str(j)+'_feat'] = np.log(np.square(test.iloc[:, i] * test.iloc[:, j]))
        
test_target = pd.DataFrame(test['TARGET'])

test = test.drop(['TARGET'], axis = 1)

test = pd.concat([test, test_], axis = 1)

# test = (test-test.mean())/test.std()

test['TARGET'] = test_target

In [27]:
test

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,0_0_feat,0_1_feat,0_2_feat,0_3_feat,0_4_feat,0_5_feat,0_6_feat,0_7_feat,0_8_feat,0_9_feat,0_10_feat,0_11_feat,0_12_feat,0_13_feat,0_14_feat,0_15_feat,0_16_feat,0_17_feat,0_18_feat,0_19_feat,0_20_feat,0_21_feat,0_22_feat,0_23_feat,0_24_feat,0_25_feat,0_26_feat,0_27_feat,0_28_feat,0_29_feat,0_30_feat,0_31_feat,0_32_feat,0_33_feat,0_34_feat,0_35_feat,0_36_feat,0_37_feat,0_38_feat,0_39_feat,0_40_feat,0_41_feat,0_42_feat,0_43_feat,0_44_feat,0_45_feat,0_46_feat,0_47_feat,0_48_feat,0_49_feat,0_50_feat,0_51_feat,0_52_feat,0_53_feat,0_54_feat,0_55_feat,0_56_feat,0_57_feat,0_58_feat,0_59_feat,0_60_feat,0_61_feat,0_62_feat,0_63_feat,0_64_feat,0_65_feat,1_0_feat,1_1_feat,1_2_feat,1_3_feat,1_4_feat,1_5_feat,1_6_feat,1_7_feat,1_8_feat,1_9_feat,1_10_feat,1_11_feat,1_12_feat,1_13_feat,1_14_feat,1_15_feat,1_16_feat,1_17_feat,...,63_49_feat,63_50_feat,63_51_feat,63_52_feat,63_53_feat,63_54_feat,63_55_feat,63_56_feat,63_57_feat,63_58_feat,63_59_feat,63_60_feat,63_61_feat,63_62_feat,63_63_feat,63_64_feat,63_65_feat,64_0_feat,64_1_feat,64_2_feat,64_3_feat,64_4_feat,64_5_feat,64_6_feat,64_7_feat,64_8_feat,64_9_feat,64_10_feat,64_11_feat,64_12_feat,64_13_feat,64_14_feat,64_15_feat,64_16_feat,64_17_feat,64_18_feat,64_19_feat,64_20_feat,64_21_feat,64_22_feat,64_23_feat,64_24_feat,64_25_feat,64_26_feat,64_27_feat,64_28_feat,64_29_feat,64_30_feat,64_31_feat,64_32_feat,64_33_feat,64_34_feat,64_35_feat,64_36_feat,64_37_feat,64_38_feat,64_39_feat,64_40_feat,64_41_feat,64_42_feat,64_43_feat,64_44_feat,64_45_feat,64_46_feat,64_47_feat,64_48_feat,64_49_feat,64_50_feat,64_51_feat,64_52_feat,64_53_feat,64_54_feat,64_55_feat,64_56_feat,64_57_feat,64_58_feat,64_59_feat,64_60_feat,64_61_feat,64_62_feat,64_63_feat,64_64_feat,64_65_feat,65_0_feat,65_1_feat,65_2_feat,65_3_feat,65_4_feat,65_5_feat,65_6_feat,65_7_feat,65_8_feat,65_9_feat,65_10_feat,65_11_feat,65_12_feat,65_13_feat,65_14_feat,65_15_feat,65_16_feat,65_17_feat,65_18_feat,65_19_feat,65_20_feat,65_21_feat,65_22_feat,65_23_feat,65_24_feat,65_25_feat,65_26_feat,65_27_feat,65_28_feat,65_29_feat,65_30_feat,65_31_feat,65_32_feat,65_33_feat,65_34_feat,65_35_feat,65_36_feat,65_37_feat,65_38_feat,65_39_feat,65_40_feat,65_41_feat,65_42_feat,65_43_feat,65_44_feat,65_45_feat,65_46_feat,65_47_feat,65_48_feat,65_49_feat,65_50_feat,65_51_feat,65_52_feat,65_53_feat,65_54_feat,65_55_feat,65_56_feat,65_57_feat,65_58_feat,65_59_feat,65_60_feat,65_61_feat,65_62_feat,65_63_feat,65_64_feat,65_65_feat,TARGET
0,16335.494480,1061.530132,41238.33840,41237.30921,70.432910,7.379175,70.376460,23229.86828,23229.78910,70.397639,7.389850,70.389234,23229.90881,25602.51370,70.835039,7.537712,70.835039,23235.72471,23235.72471,70.703960,7.467846,70.433077,23230.96180,23278.59091,70.835039,7.537712,70.441802,23238.10616,23235.72471,70.703960,7133.700113,22063.32144,4.227997e+05,2.325186e+05,11589.95489,3740.045136,1860.710635,4.138544e+06,422799.66470,22063.321440,137.249904,66395.12033,232518.55740,2.325186e+05,258.079242,30.226219,75.913405,23229.77107,23229.83301,70.376262,1473.581679,71.758838,23874.69807,23499.73762,-155.498907,7.389056,70.371005,23229.77106,12139.388420,71.758829,14.703740,84.826620,8035.667142,23254.88967,92.945298,12.071364,38.804382,33.337125,40.656438,40.656389,27.911512,23.399515,27.909909,39.508580,39.508573,27.910511,23.402406,27.910272,39.508583,39.703083,27.922899,23.442029,27.922899,39.509084,39.509084,27.919194,23.423404,27.911517,39.508674,39.512770,27.922899,23.442029,27.911765,39.509289,39.509084,27.919194,37.147362,39.405535,45.311499,44.115642,38.117979,35.855897,34.459619,49.873900,45.311499,39.405535,29.245798,41.608949,44.115642,44.115642,30.508725,26.219611,28.061378,39.508571,39.508576,27.909903,33.993094,27.948813,39.563340,39.531680,29.495469,23.402191,27.909754,39.508571,38.210613,27.948813,24.778395,28.283410,

In [28]:
# reg = Ridge(alpha=0.5).fit(pd.DataFrame(train.iloc[:, :-1]), train['TARGET'])
reg = LinearRegression().fit(pd.DataFrame(train.iloc[:90000, :-1]), train['TARGET'][:90000])

print(reg.score(pd.DataFrame(train.iloc[:90000, :-1]), train['TARGET'][:90000]))

train_pred = reg.predict(pd.DataFrame(train.iloc[:, :-1]))

test_pred = reg.predict(pd.DataFrame(test.iloc[:, :-1]))

train_res = np.sum(train['TARGET'][train_pred>0])
test_res = np.sum(test['TARGET'][test_pred>0])

0.008533228061671227


In [29]:
1-np.sum(np.square(train['TARGET']-train_pred))/np.sum(np.square(train['TARGET']-np.mean(train['TARGET'])))

0.008533228061671227

In [30]:
print(train_res/train_max)
print(test_res/test_max)

0.12273172711571358
0.05080486487823355


In [31]:
train_ = pd.DataFrame()

for i in range(66, 100):
    for j in range(66, 100):
        train_[str(i)+'_'+str(j)+'_feat'] = np.log(train.iloc[:, i] * train.iloc[:, j])
        
train_target = pd.DataFrame(train['TARGET'])

train = train.drop(['TARGET'], axis = 1)

train = pd.concat([train, train_], axis = 1)

# train = (train-train.mean())/train.std()

train['TARGET'] = train_target

In [32]:
train

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,0_0_feat,0_1_feat,0_2_feat,0_3_feat,0_4_feat,0_5_feat,0_6_feat,0_7_feat,0_8_feat,0_9_feat,0_10_feat,0_11_feat,0_12_feat,0_13_feat,0_14_feat,0_15_feat,0_16_feat,0_17_feat,0_18_feat,0_19_feat,0_20_feat,0_21_feat,0_22_feat,0_23_feat,0_24_feat,0_25_feat,0_26_feat,0_27_feat,0_28_feat,0_29_feat,0_30_feat,0_31_feat,0_32_feat,0_33_feat,0_34_feat,0_35_feat,0_36_feat,0_37_feat,0_38_feat,0_39_feat,0_40_feat,0_41_feat,0_42_feat,0_43_feat,0_44_feat,0_45_feat,0_46_feat,0_47_feat,0_48_feat,0_49_feat,0_50_feat,0_51_feat,0_52_feat,0_53_feat,0_54_feat,0_55_feat,0_56_feat,0_57_feat,0_58_feat,0_59_feat,0_60_feat,0_61_feat,0_62_feat,0_63_feat,0_64_feat,0_65_feat,1_0_feat,1_1_feat,1_2_feat,1_3_feat,1_4_feat,1_5_feat,1_6_feat,1_7_feat,1_8_feat,1_9_feat,1_10_feat,1_11_feat,1_12_feat,1_13_feat,1_14_feat,1_15_feat,1_16_feat,1_17_feat,...,95_87_feat,95_88_feat,95_89_feat,95_90_feat,95_91_feat,95_92_feat,95_93_feat,95_94_feat,95_95_feat,95_96_feat,95_97_feat,95_98_feat,95_99_feat,96_66_feat,96_67_feat,96_68_feat,96_69_feat,96_70_feat,96_71_feat,96_72_feat,96_73_feat,96_74_feat,96_75_feat,96_76_feat,96_77_feat,96_78_feat,96_79_feat,96_80_feat,96_81_feat,96_82_feat,96_83_feat,96_84_feat,96_85_feat,96_86_feat,96_87_feat,96_88_feat,96_89_feat,96_90_feat,96_91_feat,96_92_feat,96_93_feat,96_94_feat,96_95_feat,96_96_feat,96_97_feat,96_98_feat,96_99_feat,97_66_feat,97_67_feat,97_68_feat,97_69_feat,97_70_feat,97_71_feat,97_72_feat,97_73_feat,97_74_feat,97_75_feat,97_76_feat,97_77_feat,97_78_feat,97_79_feat,97_80_feat,97_81_feat,97_82_feat,97_83_feat,97_84_feat,97_85_feat,97_86_feat,97_87_feat,97_88_feat,97_89_feat,97_90_feat,97_91_feat,97_92_feat,97_93_feat,97_94_feat,97_95_feat,97_96_feat,97_97_feat,97_98_feat,97_99_feat,98_66_feat,98_67_feat,98_68_feat,98_69_feat,98_70_feat,98_71_feat,98_72_feat,98_73_feat,98_74_feat,98_75_feat,98_76_feat,98_77_feat,98_78_feat,98_79_feat,98_80_feat,98_81_feat,98_82_feat,98_83_feat,98_84_feat,98_85_feat,98_86_feat,98_87_feat,98_88_feat,98_89_feat,98_90_feat,98_91_feat,98_92_feat,98_93_feat,98_94_feat,98_95_feat,98_96_feat,98_97_feat,98_98_feat,98_99_feat,99_66_feat,99_67_feat,99_68_feat,99_69_feat,99_70_feat,99_71_feat,99_72_feat,99_73_feat,99_74_feat,99_75_feat,99_76_feat,99_77_feat,99_78_feat,99_79_feat,99_80_feat,99_81_feat,99_82_feat,99_83_feat,99_84_feat,99_85_feat,99_86_feat,99_87_feat,99_88_feat,99_89_feat,99_90_feat,99_91_feat,99_92_feat,99_93_feat,99_94_feat,99_95_feat,99_96_feat,99_97_feat,99_98_feat,99_99_feat,TARGET
0,5874.524387,1072.671848,41440.76212,41440.23732,70.405148,7.392780,70.377281,23229.69262,23229.72655,70.378864,7.389173,70.380160,23229.76782,23379.81637,83.418623,11.615135,83.418623,23466.72590,23466.72590,83.418623,7.425618,70.494241,23229.77107,23229.77107,70.376262,7.389056,70.376262,23229.77107,23229.77107,70.376262,4898.757333,15165.927590,297487.1654,297487.16540,15165.927590,4898.757333,70.376262,2.322977e+04,23229.77107,70.376262,7.389056,70.376262,23229.77107,23229.77107,64.192982,20.940618,76.070270,23376.73707,23343.13291,77.290965,347.308164,102.380501,24823.08137,24120.94894,332.757607,17.386711,129.622187,23936.99077,21670.192330,71.518948,11.399004,78.006816,26437.161240,23811.09670,141.997532,22.474794,34.713522,31.312576,38.620801,38.620776,25.865294,21.357769,25.864502,37.463134,37.463137,25.864547,21.356792,25.864584,37.463140,37.476018,26.204504,22.261379,26.204504,37.483438,37.483438,26.204504,21.366633,25.867823,37.463141,37.463141,25.864473,21.356761,25.864473,37.463141,37.463141,25.864473,34.350234,36.610374,42.563013,42.563013,36.610374,34.350234,25.864473,37.463141,37.463141,25.864473,21.356761,25.864473,37.463141,37.463141,25.680549,23.440142,26.020076,37.475754,37.472877,26.051915,29.057186,26.614153,37.595819,37.538433,28.971589,23.068173,27.086009,37.523121,37.324147,25.896686,22.223813

In [33]:
test_ = pd.DataFrame()

for i in range(66, 100):
    for j in range(66, 100):
        test_[str(i)+'_'+str(j)+'_feat'] = np.log(test.iloc[:, i] * test.iloc[:, j])
        
test_target = pd.DataFrame(test['TARGET'])

test = test.drop(['TARGET'], axis = 1)

test = pd.concat([test, test_], axis = 1)

# test = (test-test.mean())/test.std()

test['TARGET'] = test_target

In [34]:
test

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,0_0_feat,0_1_feat,0_2_feat,0_3_feat,0_4_feat,0_5_feat,0_6_feat,0_7_feat,0_8_feat,0_9_feat,0_10_feat,0_11_feat,0_12_feat,0_13_feat,0_14_feat,0_15_feat,0_16_feat,0_17_feat,0_18_feat,0_19_feat,0_20_feat,0_21_feat,0_22_feat,0_23_feat,0_24_feat,0_25_feat,0_26_feat,0_27_feat,0_28_feat,0_29_feat,0_30_feat,0_31_feat,0_32_feat,0_33_feat,0_34_feat,0_35_feat,0_36_feat,0_37_feat,0_38_feat,0_39_feat,0_40_feat,0_41_feat,0_42_feat,0_43_feat,0_44_feat,0_45_feat,0_46_feat,0_47_feat,0_48_feat,0_49_feat,0_50_feat,0_51_feat,0_52_feat,0_53_feat,0_54_feat,0_55_feat,0_56_feat,0_57_feat,0_58_feat,0_59_feat,0_60_feat,0_61_feat,0_62_feat,0_63_feat,0_64_feat,0_65_feat,1_0_feat,1_1_feat,1_2_feat,1_3_feat,1_4_feat,1_5_feat,1_6_feat,1_7_feat,1_8_feat,1_9_feat,1_10_feat,1_11_feat,1_12_feat,1_13_feat,1_14_feat,1_15_feat,1_16_feat,1_17_feat,...,95_87_feat,95_88_feat,95_89_feat,95_90_feat,95_91_feat,95_92_feat,95_93_feat,95_94_feat,95_95_feat,95_96_feat,95_97_feat,95_98_feat,95_99_feat,96_66_feat,96_67_feat,96_68_feat,96_69_feat,96_70_feat,96_71_feat,96_72_feat,96_73_feat,96_74_feat,96_75_feat,96_76_feat,96_77_feat,96_78_feat,96_79_feat,96_80_feat,96_81_feat,96_82_feat,96_83_feat,96_84_feat,96_85_feat,96_86_feat,96_87_feat,96_88_feat,96_89_feat,96_90_feat,96_91_feat,96_92_feat,96_93_feat,96_94_feat,96_95_feat,96_96_feat,96_97_feat,96_98_feat,96_99_feat,97_66_feat,97_67_feat,97_68_feat,97_69_feat,97_70_feat,97_71_feat,97_72_feat,97_73_feat,97_74_feat,97_75_feat,97_76_feat,97_77_feat,97_78_feat,97_79_feat,97_80_feat,97_81_feat,97_82_feat,97_83_feat,97_84_feat,97_85_feat,97_86_feat,97_87_feat,97_88_feat,97_89_feat,97_90_feat,97_91_feat,97_92_feat,97_93_feat,97_94_feat,97_95_feat,97_96_feat,97_97_feat,97_98_feat,97_99_feat,98_66_feat,98_67_feat,98_68_feat,98_69_feat,98_70_feat,98_71_feat,98_72_feat,98_73_feat,98_74_feat,98_75_feat,98_76_feat,98_77_feat,98_78_feat,98_79_feat,98_80_feat,98_81_feat,98_82_feat,98_83_feat,98_84_feat,98_85_feat,98_86_feat,98_87_feat,98_88_feat,98_89_feat,98_90_feat,98_91_feat,98_92_feat,98_93_feat,98_94_feat,98_95_feat,98_96_feat,98_97_feat,98_98_feat,98_99_feat,99_66_feat,99_67_feat,99_68_feat,99_69_feat,99_70_feat,99_71_feat,99_72_feat,99_73_feat,99_74_feat,99_75_feat,99_76_feat,99_77_feat,99_78_feat,99_79_feat,99_80_feat,99_81_feat,99_82_feat,99_83_feat,99_84_feat,99_85_feat,99_86_feat,99_87_feat,99_88_feat,99_89_feat,99_90_feat,99_91_feat,99_92_feat,99_93_feat,99_94_feat,99_95_feat,99_96_feat,99_97_feat,99_98_feat,99_99_feat,TARGET
0,16335.494480,1061.530132,41238.33840,41237.30921,70.432910,7.379175,70.376460,23229.86828,23229.78910,70.397639,7.389850,70.389234,23229.90881,25602.51370,70.835039,7.537712,70.835039,23235.72471,23235.72471,70.703960,7.467846,70.433077,23230.96180,23278.59091,70.835039,7.537712,70.441802,23238.10616,23235.72471,70.703960,7133.700113,22063.32144,4.227997e+05,2.325186e+05,11589.95489,3740.045136,1860.710635,4.138544e+06,422799.66470,22063.321440,137.249904,66395.12033,232518.55740,2.325186e+05,258.079242,30.226219,75.913405,23229.77107,23229.83301,70.376262,1473.581679,71.758838,23874.69807,23499.73762,-155.498907,7.389056,70.371005,23229.77106,12139.388420,71.758829,14.703740,84.826620,8035.667142,23254.88967,92.945298,12.071364,38.804382,33.337125,40.656438,40.656389,27.911512,23.399515,27.909909,39.508580,39.508573,27.910511,23.402406,27.910272,39.508583,39.703083,27.922899,23.442029,27.922899,39.509084,39.509084,27.919194,23.423404,27.911517,39.508674,39.512770,27.922899,23.442029,27.911765,39.509289,39.509084,27.919194,37.147362,39.405535,45.311499,44.115642,38.117979,35.855897,34.459619,49.873900,45.311499,39.405535,29.245798,41.608949,44.115642,44.115642,30.508725,26.219611,28.061378,39.508571,39.508576,27.909903,33.993094,27.948813,39.563340,39.531680,29.495469,23.402191,27.909754,39.508571,38.210613,27.948813,

In [35]:
# reg = Ridge(alpha=0.5).fit(pd.DataFrame(train.iloc[:, :-1]), train['TARGET'])
reg = LinearRegression().fit(pd.DataFrame(train.iloc[:90000, :-1]), train['TARGET'][:90000])
print(reg.score(pd.DataFrame(train.iloc[:90000, :-1]), train['TARGET'][:90000]))

train_pred = reg.predict(pd.DataFrame(train.iloc[:, :-1]))

test_pred = reg.predict(pd.DataFrame(test.iloc[:, :-1]))

train_res = np.sum(train['TARGET'][train_pred>0])
test_res = np.sum(test['TARGET'][test_pred>0])

0.010157524225542724


In [36]:
1-np.sum(np.square(train['TARGET']-train_pred))/np.sum(np.square(train['TARGET']-np.mean(train['TARGET'])))

0.010157524225542724

In [37]:
print(train_res/train_max)
print(test_res/test_max)

0.12414569051086169
0.05370961720148238


In [38]:
train_ = train.drop('TARGET', axis=1)
train_ = (train_-train_.mean())/train_.std()

test_ = test.drop('TARGET', axis=1)
test_ = (test_-test_.mean())/test_.std()

In [39]:
train_std = pd.concat([train_, train['TARGET']], axis=1)
test_std = pd.concat([test_, test['TARGET']], axis=1)

In [40]:
reg = Ridge(alpha=0.5).fit(pd.DataFrame(train_std.iloc[:, :-1]), train_std['TARGET'])
train_pred = reg.predict(pd.DataFrame(train_std.iloc[:, :-1]))

test_pred = reg.predict(pd.DataFrame(test_std.iloc[:, :-1]))

train_res = np.sum(train_std['TARGET'][train_pred>0])
test_res = np.sum(test_std['TARGET'][test_pred>0])

In [41]:
print(train_res)
print(test_res)

23.766619693800003
-0.08101852779999977
